# Testing new code to split amps into regions in PTC.

Craig Lage 13-Feb-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from lsst.cp.pipe.ptc import PlotPhotonTransferCurveTask, PlotPhotonTransferCurveConfig
%matplotlib inline


In [ ]:
detector=55
expId = 3021120700188
fullButler = Butler("/repo/main", collections=["u/cslage/bps_13144S"])
fullPtc = fullButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
basePtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_baseline_12feb23"])
basePtc = basePtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
reg_1_PtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_1_12feb23"])
reg_1_Ptc = reg_1_PtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
reg_2_PtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_2_12feb23"])
reg_2_Ptc = reg_2_PtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
reg_4_PtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_4_12feb23"])
reg_4_Ptc = reg_4_PtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
reg_8_PtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_8C_12feb23"])
reg_8_Ptc = reg_8_PtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
reg_16_PtcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_16_12feb23"])
reg_16_Ptc = reg_16_PtcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")

exp = fullButler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
#camera = fullButler.get('camera', instrument='LSSTCam')

In [ ]:
for amp in exp.getDetector().getAmplifiers():
    print(amp.getName(), basePtc.gain[amp.getName()], reg_1_Ptc.gain[amp.getName()], reg_2_Ptc.gain[amp.getName()],\
         reg_4_Ptc.gain[amp.getName()], reg_8_Ptc.gain[amp.getName()], reg_16_Ptc.gain[amp.getName()])

In [ ]:
ampName = 'C00'
fig = plt.figure(figsize=(8,8))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Impact of PTC Regions \nRun 13144, Detector:{detector}, amp:{ampName}")
plotCounter = 1
names = ['1x1', '1x2', '1x4', '2x4']
for i, ptc in enumerate([reg_1_Ptc, reg_2_Ptc, reg_4_Ptc, reg_8_Ptc]):
    gain_diff = (ptc.gain[ampName] - basePtc.gain[ampName]) / basePtc.gain[ampName] * 100.0
    plt.subplot(2,2,plotCounter)
    plt.scatter(ptc.rawMeans[ampName], ptc.rawVars[ampName], label=names[i], marker='o', color='blue')
    plt.scatter(basePtc.rawMeans[ampName], basePtc.rawVars[ampName], marker='x', s=200, color='red', label="Baseline")
    plt.plot(basePtc.finalMeans[ampName], basePtc.finalModelVars[ampName], color='red', label="Baseline")
    plt.text(40100, 23000, f"Gain diff from base = {gain_diff:.4f}%")
    plt.xlim(40000,45000)
    plt.ylim(22000,27000)
    plt.xlabel("Flux(ADU)")
    plt.ylabel("Variance(ADU^2)")
    plt.legend()
    plotCounter += 1
plt.savefig("/home/c/cslage/u/BOT_LSSTCam/isr_challenge/PTC_Regions_C00_13Feb23.png")



In [ ]:
pdf = PdfPages("/home/c/cslage/u/BOT_LSSTCam/isr_challenge/PTC_Regions_13Feb23.png")
for amp in exp.getDetector().getAmplifiers():
    ampName = amp.getName()
    fig = plt.figure(figsize=(8,8))
    plt.subplots_adjust(wspace=0.5)
    plt.suptitle(f"Impact of PTC Regions \nRun 13144, Detector:{detector}, amp:{ampName}")
    plotCounter = 1
    names = ['1x1', '1x2', '1x4', '2x4']
    for i, ptc in enumerate([reg_1_Ptc, reg_2_Ptc, reg_4_Ptc, reg_8_Ptc]):
        gain_diff = (ptc.gain[ampName] - basePtc.gain[ampName]) / basePtc.gain[ampName] * 100.0
        plt.subplot(2,2,plotCounter)
        plt.scatter(ptc.rawMeans[ampName], ptc.rawVars[ampName], label=names[i], marker='o', color='blue')
        plt.scatter(basePtc.rawMeans[ampName], basePtc.rawVars[ampName], marker='x', s=200, color='red', label="Baseline")
        plt.plot(basePtc.finalMeans[ampName], basePtc.finalModelVars[ampName], color='red', label="Baseline")
        plt.text(40100, 23000, f"Gain diff from base = {gain_diff:.4f}%")
        plt.xlim(40000,45000)
        plt.ylim(22000,27000)
        plt.xlabel("Flux(ADU)")
        plt.ylabel("Variance(ADU^2)")
        plt.legend()
        plotCounter += 1
        pdf.savefig(fig)  # saves the current figure into a pdf page
        #plt.close()
pdf.close()


In [ ]:
names = ['1x1', '1x2', '1x4', '2x4', '2x8']
for detector in [55]:
    fig = plt.figure(figsize=(8,8))
    plt.subplots_adjust(hspace=0.2, wspace=0.1)
    plt.suptitle(f"Gain difference regions {detector}", fontsize=24)
    plotCounter = 1
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        diffs = []
        xaxis = [1,2,3,4,5]
        for i, ptc in enumerate([reg_1_Ptc, reg_2_Ptc, reg_4_Ptc, reg_8_Ptc, reg_16_Ptc]):
            gain_diff = (ptc.gain[ampName] - basePtc.gain[ampName]) / basePtc.gain[ampName] * 100.0
            diffs.append(gain_diff)
        plt.subplot(4,4,plotCounter)
        plt.title(ampName)
        plt.scatter(xaxis, diffs)
        plt.plot([1,5], [0.01,0.01], ls='--', color='red')
        plt.plot([1,5], [-0.01,-0.01], ls='--', color='red')
        plt.ylim(-0.05, 0.05)
        plotCounter += 1
        if plotCounter%4 == 2:
            plt.yticks([-0.05, 0.0,0.05])
            plt.ylabel("Correction(%)")
        else:
            plt.yticks([])
        if plotCounter > 13:
            plt.xticks([1,2,3,4,5],names)
            plt.xlabel("Flux")
        else:
            plt.xticks([])
    plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Gains_Regions_{detector}_25Jan23.png")
